In [37]:
import numpy as np
import pandas as pd
import datetime, os
#import randomization # this one doesn't make a dataframe :(
# from stochatreat import stochatreat # this one doesnt include the block_id :(
from rpy2.robjects.packages import importr
import rpy2.robjects as ro # this will be useful if not running in Jupyter
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [38]:
n_randomizations = 100_000
n_arms = 3
units_per_block = 60 #aka total number of randomizations in each block. assumes balanced blocks, so block size = units_per_block = n_arms * (x arm appearances)

In [39]:
%%R -i n_randomizations -i n_arms -i units_per_block
library(blockrand)
set.seed(1471122187) # this is important

########### French Wikipedia Welcome Experiment
#observations = 10
## We will run this for 90 days. In 2019 The maximum number of newcomers per 30 days has never exceeded
## 20,000. So 60,000 should be sufficient.

frwiki <- blockrand(n=n_randomizations, 
                    num.levels = n_arms,
                    levels = c(0,1,2),
                    block.sizes = c(units_per_block/n_arms, units_per_block/n_arms, units_per_block/n_arms), 
                    id.prefix='newcomer_', 
                    #block.prefix='frwiki.block',
                    #stratum='post'
                    )

In [40]:
%R -o frwiki

In [41]:
frwiki.head(10)

,id,block.id,block.size,treatment
1,newcomer_000001,1,60.0,1
2,newcomer_000002,1,60.0,2
3,newcomer_000003,1,60.0,2
4,newcomer_000004,1,60.0,0
5,newcomer_000005,1,60.0,2
6,newcomer_000006,1,60.0,0
7,newcomer_000007,1,60.0,0
8,newcomer_000008,1,60.0,1
9,newcomer_000009,1,60.0,1
10,newcomer_000010,1,60.0,2


In [42]:
target_colnames = {"block.id":"randomization_block_id",
                       "block.size":"randomization_block_size",
                       "treatment":"randomization_arm"}
exportable = frwiki.rename(target_colnames,
                      axis='columns')

In [43]:
exportable = exportable[["randomization_arm", "randomization_block_id",]]

In [44]:
exportable = exportable.astype(int)

In [45]:
exportable.head()

,randomization_arm,randomization_block_id
1,1,1
2,2,1
3,2,1
4,0,1
5,2,1


In [58]:
exportable['randomization_arm'][:60].sum()

60

In [62]:
fname_stem = 'fr-randomziation'
date = datetime.datetime.today().strftime('%Y%m%d')
fname = os.path.join('/data/project/frwelcome/randomizations', f'{fname_stem}-{date}-{n_randomizations}-{units_per_block}upb.split.noindex.json')
exportable.to_json(fname, orient='split', index=False)

In [64]:
!du -h /data/project/frwelcome/randomizations/*

948K	/data/project/frwelcome/randomizations/fr-dummy-randomziation-20190924-100000.split.noindex.json
4.0K	/data/project/frwelcome/randomizations/fr-obfuscations.json
4.0K	/data/project/frwelcome/randomizations/fr-randomziation-20191014-10.split.noindex.json
816K	/data/project/frwelcome/randomizations/fr-randomziation-20191015-100000-60upb.split.noindex.json
